In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# Checkout sessions include all sessions
# flag_score = True for Scoring
# For scoring, only keep pre-checkout sessions which do not reach checkout.
# For training, we need all pre-checkout sessions.

flag_score = False
checkout_file = '20200222_0223_checkout_sessions.csv'

In [3]:
# Production Code

df = pd.read_csv(checkout_file)
df.rename(columns={i:i.lower() for i in df.columns.tolist()}, 
         inplace=True)
df_BAK = df.copy()

In [4]:
df_multiple_sessions = df[['date', 'zone_id', 'warehouse_location_id', 'user_id', 'visit_token']]\
    .groupby(['date', 'zone_id', 'warehouse_location_id', 'user_id'])\
    .count().reset_index()

In [5]:
df_multiple_sessions[df_multiple_sessions['visit_token'] >= 5].head(10)

,date,zone_id,warehouse_location_id,user_id,visit_token
1018,2020-02-22,166,3656,57625844,5
1904,2020-02-22,186,3417,47321835,5
2231,2020-02-22,206,4053,30562289,6
2666,2020-02-22,226,4246,79116085,5
2748,2020-02-22,244,4753,75735020,6
3796,2020-02-22,354,7508,29658836,5
3804,2020-02-22,354,7508,45023419,5
3871,2020-02-22,357,7620,72381703,5
4619,2020-02-22,413,23638,79403585,5
5457,2020-02-22,477,18414,68363779,5


In [6]:
explore_user_id = 30562289
explore_zone_id = 206

In [7]:
df[(df['date'] == '2020-02-22') & (df['zone_id'] == explore_zone_id) & 
   (df['user_id'] == explore_user_id)]

,date,zone_id,warehouse_location_id,user_id,visit_token,first_option_time_local,first_option_ind,first_wait_time_at_event,second_option_time_local,second_option_ind,...,fourth_wait_time_at_event,fifth_option_time_local,fifth_option_ind,fifth_wait_time_at_event,express_member_ind,order_items_cnt,placed_order,session_start_time,session_end_time,num_event_in_session
8116,2020-02-22,206,4053,30562289,f3173a6c-6038-4c79-ac11-29570c170fd1,2020-02-22 21:00:00.000 -0600,Y,88.0,2020-02-23 09:00:00.000 -0600,Y,...,928.0,2020-02-23 12:00:00.000 -0600,Y,988.0,NaN,23,0,2020-02-22 19:13:53.544,2020-02-22 19:32:09.577,16
8117,2020-02-22,206,4053,30562289,f5b8639b-bfda-4a9e-a7d4-9933c054104d,2020-02-22 13:00:00.000 -0600,Y,120.0,2020-02-22 14:00:00.000 -0600,Y,...,300.0,2020-02-22 17:00:00.000 -0600,Y,360.0,NaN,26,0,2020-02-22 10:54:50.657,2020-02-22 11:00:39.253,13
9503,2020-02-22,206,4053,30562289,981dd838-d2a5-40b3-841c-90d7b0fdabee,2020-02-22 21:00:00.000 -0600,Y,97.0,2020-02-23 09:00:00.000 -0600,Y,...,937.0,2020-02-23 12:00:00.000 -0600,Y,997.0,NaN,29,0,2020-02-22 19:17:21.488,2020-02-22 19:23:34.968,8
14217,2020-02-22,206,4053,30562289,31b85fe1-d1e4-4eb9-9450-2138f2fe2b74,2020-02-22 14:00:00.000 -0600,N,99.0,2020-02-22 15:00:00.000 -0600,N,...,279.0,2020-02-22 18:00:00.000 -0600,N,339.0,NaN,26,0,2020-02-22 12:21:53.866,2020-02-22 12:21:53.910,2
15851,2020-02-22,206,4053,30562289,1ec0d222-9587-4862-a9ff-a2e2896492e9,2020-02-22 15:00:00.000 -0600,Y,84.0,2020-02-22 16:00:00.000 -0600,Y,...,264.0,2020-02-22 19:00:00.000 -0600,Y,324.0,NaN,30,0,2020-02-22 13:08:16.663,2020-02-22 13:36:55.691,18
24901,2020-02-22,206,4053,30562289,bb6452c0-9a8a-43e3-a1c9-c4d3980ca262,2020-02-22 21:00:00.000 -0600,Y,75.0,2020-02-23 09:00:00.000 -0600,Y,...,915.0,2020-02-23 12:00:00.000 -0600,Y,975.0,NaN,23,1,2020-02-22 19:32:44.320,2020-02-22 20:01:36.053,14


In [8]:
df.columns

Index(['date', 'zone_id', 'warehouse_location_id', 'user_id', 'visit_token',
       'first_option_time_local', 'first_option_ind',
       'first_wait_time_at_event', 'second_option_time_local',
       'second_option_ind', 'second_wait_time_at_event',
       'third_option_time_local', 'third_option_ind',
       'third_wait_time_at_event', 'fourth_option_time_local',
       'fourth_option_ind', 'fourth_wait_time_at_event',
       'fifth_option_time_local', 'fifth_option_ind',
       'fifth_wait_time_at_event', 'express_member_ind', 'order_items_cnt',
       'placed_order', 'session_start_time', 'session_end_time',
       'num_event_in_session'],
      dtype='object')

In [9]:
# Production Code

df['session_start_time'] = df['session_start_time'].map(lambda x: x[:19])
df['session_start_time'] = pd.to_datetime(df['session_start_time'], format="%Y-%m-%dT%H:%M:%S")

df['session_end_time'] = df['session_end_time'].map(lambda x: x[:19])
df['session_end_time'] = pd.to_datetime(df['session_end_time'], format="%Y-%m-%dT%H:%M:%S")

In [10]:
df.head(3)

,date,zone_id,warehouse_location_id,user_id,visit_token,first_option_time_local,first_option_ind,first_wait_time_at_event,second_option_time_local,second_option_ind,...,fourth_wait_time_at_event,fifth_option_time_local,fifth_option_ind,fifth_wait_time_at_event,express_member_ind,order_items_cnt,placed_order,session_start_time,session_end_time,num_event_in_session
0,2020-02-22,201,3759,32601463,6dac932a-13f8-4599-917a-f74df51290f8,2020-02-22 11:00:00.000 -0500,Y,104.0,2020-02-22 12:00:00.000 -0500,Y,...,284.0,2020-02-22 15:00:00.000 -0500,Y,344.0,False,29,1,2020-02-22 09:15:38,2020-02-22 09:16:41,5
1,2020-02-22,43,634,74736585,cf994f44-6ded-43df-b41a-89e34309299b,2020-02-22 16:00:00.000 -0500,Y,93.0,2020-02-22 17:00:00.000 -0500,Y,...,273.0,2020-02-22 20:00:00.000 -0500,Y,333.0,False,14,1,2020-02-22 14:07:00,2020-02-22 14:27:24,8
2,2020-02-22,1163,26214,39543936,45e6d0aa-3a1a-4037-95bb-9af3fc060ebc,2020-02-22 12:00:00.000 -0500,Y,68.0,2020-02-22 13:00:00.000 -0500,Y,...,248.0,2020-02-22 16:00:00.000 -0500,Y,308.0,False,26,1,2020-02-22 10:04:18,2020-02-22 10:58:56,15


In [11]:
# Production Code

# One session join with every other session
# Get sessions ended before session started for every session
df_before = df[['date', 'zone_id', 'warehouse_location_id', 'user_id', 'visit_token',
    'session_start_time']]\
    .merge(df[['date', 'zone_id', 'warehouse_location_id', 'user_id', 'visit_token',
    'session_end_time', 'placed_order', 'num_event_in_session']]\
    .rename(columns={'visit_token':'visit_token_joined',
                    'session_end_time':'session_end_time_joined'}),
    on=['date', 'zone_id', 'warehouse_location_id', 'user_id'],
    how='left')
df_before = df_before[df_before['session_end_time_joined']
                                   < df_before['session_start_time']]
df_before = df_before.groupby(['date', 'zone_id', 'warehouse_location_id',
                'session_start_time', 'user_id', 'visit_token'])\
    .sum().reset_index().rename(columns={'placed_order':'placed_order_before',
                                        'num_event_in_session':'num_event_before'})

# One session join with every other session
# Get sessions started after session ended for every session
df_after = df[['date', 'zone_id', 'warehouse_location_id', 'user_id', 'visit_token',
    'session_end_time']]\
    .merge(df[['date', 'zone_id', 'warehouse_location_id', 'user_id', 'visit_token',
    'session_start_time', 'placed_order', 'num_event_in_session']]\
    .rename(columns={'visit_token':'visit_token_joined',
                    'session_start_time':'session_start_time_joined'}),
    on=['date', 'zone_id', 'warehouse_location_id', 'user_id'],
    how='left')
df_after = df_after[df_after['session_start_time_joined']
                                   > df_after['session_end_time']]
df_after = df_after.groupby(['date', 'zone_id', 'warehouse_location_id',
                'user_id', 'session_end_time', 'visit_token'])\
    .sum().reset_index().rename(columns={'placed_order':'placed_order_after',
                                        'num_event_in_session':'num_event_after'})

# Join df_before and df_after
df = df.merge(df_before[['date', 'zone_id', 'warehouse_location_id',
                        'user_id', 'visit_token', 'placed_order_before',
                       'num_event_before']],
            on=['date', 'zone_id', 'warehouse_location_id',
                        'user_id', 'visit_token'],
            how='left')
df = df.merge(df_after[['date', 'zone_id', 'warehouse_location_id',
                        'user_id', 'visit_token', 'placed_order_after',
                       'num_event_after']],
            on=['date', 'zone_id', 'warehouse_location_id',
                        'user_id', 'visit_token'],
            how='left')

# Fill in NA num_event_after and num_event_before
df.fillna(0, inplace=True)

# Get session length
df['session_length'] = (df['session_end_time'] - df['session_start_time']).map(lambda x: x.seconds / 60)

In [12]:
df[df['user_id'] == explore_user_id].sort_values(by=['session_start_time'])

,date,zone_id,warehouse_location_id,user_id,visit_token,first_option_time_local,first_option_ind,first_wait_time_at_event,second_option_time_local,second_option_ind,...,order_items_cnt,placed_order,session_start_time,session_end_time,num_event_in_session,placed_order_before,num_event_before,placed_order_after,num_event_after,session_length
8117,2020-02-22,206,4053,30562289,f5b8639b-bfda-4a9e-a7d4-9933c054104d,2020-02-22 13:00:00.000 -0600,Y,120.0,2020-02-22 14:00:00.000 -0600,Y,...,26,0,2020-02-22 10:54:50,2020-02-22 11:00:39,13,0.0,0.0,1.0,58.0,5.816667
14217,2020-02-22,206,4053,30562289,31b85fe1-d1e4-4eb9-9450-2138f2fe2b74,2020-02-22 14:00:00.000 -0600,N,99.0,2020-02-22 15:00:00.000 -0600,N,...,26,0,2020-02-22 12:21:53,2020-02-22 12:21:53,2,0.0,13.0,1.0,56.0,0.000000
15851,2020-02-22,206,4053,30562289,1ec0d222-9587-4862-a9ff-a2e2896492e9,2020-02-22 15:00:00.000 -0600,Y,84.0,2020-02-22 16:00:00.000 -0600,Y,...,30,0,2020-02-22 13:08:16,2020-02-22 13:36:55,18,0.0,15.0,1.0,38.0,28.650000
8116,2020-02-22,206,4053,30562289,f3173a6c-6038-4c79-ac11-29570c170fd1,2020-02-22 21:00:00.000 -0600,Y,88.0,2020-02-23 09:00:00.000 -0600,Y,...,23,0,2020-02-22 19:13:53,2020-02-22 19:32:09,16,0.0,33.0,1.0,14.0,18.266667
9503,2020-02-22,206,4053,30562289,981dd838-d2a5-40b3-841c-90d7b0fdabee,2020-02-22 21:00:00.000 -0600,Y,97.0,2020-02-23 09:00:00.000 -0600,Y,...,29,0,2020-02-22 19:17:21,2020-02-22 19:23:34,8,0.0,33.0,1.0,14.0,6.216667
24901,2020-02-22,206,4053,30562289,bb6452c0-9a8a-43e3-a1c9-c4d3980ca262,2020-02-22 21:00:00.000 -0600,Y,75.0,2020-02-23 09:00:00.000 -0600,Y,...,23,1,2020-02-22 19:32:44,2020-02-22 20:01:36,14,0.0,57.0,0.0,0.0,28.866667
